In [ ]:
import os
import time
import torch
import torch.optim as optim
import torch.nn as nn
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torchvision.models import alexnet

In [ ]:
model = alexnet()
convs = []
# list of a conv layers within alexnn
for i,k in enumerate(model.modules()):
    if isinstance(k, nn.Conv2d):
        convs.append(k)

In [ ]:
l1 = convs[0].weight

In [ ]:
# Returns the covariance matrix of m
def cov(m, rowvar=False):
    if m.dim() > 2:
        raise ValueError('m has more than 2 dimensions')
    if m.dim() < 2:
        m = m.view(1, -1)
    if not rowvar and m.size(0) != 1:
        m = m.t()
    # m = m.type(torch.double)  # uncomment this line if desired
    fact = 1.0 / (m.size(1) - 1)
    m -= torch.mean(m, dim=1, keepdim=True)
    mt = m.t()  # if complex: mt = m.t().conj()
    return fact * m.matmul(mt).squeeze()

In [ ]:
def _mahalanobis(X):
    VI = torch.inverse(cov(X)) #covariance matrix
    total_dist = 0
    for i,v in enumerate(X):
        dist = 0
        for j,u in enumerate(X):
            if i == j:
                continue
            x = (v-u).unsqueeze(0).t()
            y = (v - u).unsqueeze(0)
            
            dist = (torch.mm(torch.mm(y,VI),x)) #sqrt of dist returns NaN (?)
            
            total_dist +=dist
            print(dist)
    return total_dist

In [ ]:

for j in range(l1.shape[1]): # iterate over channels
    X = torch.reshape(l1[0][j], [1,l1[0][j].shape[1]**2])  # gets first elem
    for i,w in enumerate(l1): # iterates over filters
        if i == 0:
            continue
        else:
            y = torch.reshape(w[0], [1,w[j].shape[1]**2])
            X = torch.cat((X,y))
    dist = _mahalanobis(X)

In [ ]:
#x = X[0] #vector1
#y = X[2] #vector 2
### mahalanobis params
#VI = torch.inverse(cov(X)) #inverse of covariance matrix
#p2 = (x - y).unsqueeze(0)
#p1 = (x-y).unsqueeze(0).t()

In [ ]:
#_mahalanobis(X)

In [ ]:
X.shape